### Домашнее задание по теме «Рекомендации на основе содержания»

In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer

In [21]:
links = pd.read_csv('ml-latest/links.csv')
movies = pd.read_csv('ml-latest/movies.csv')
ratings = pd.read_csv('ml-latest/ratings.csv')
tags = pd.read_csv('ml-latest/tags.csv')

In [22]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [23]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [24]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [25]:
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


Будем предсказывать рейтинг для пользователя **1**

In [26]:
df_user = ratings.merge(movies, on='movieId', how='left')[ratings['userId'] == 1]

In [27]:
df_user.shape

(16, 6)

In [28]:
df_user.head()

,userId,movieId,rating,timestamp,title,genres
0,1,307,3.5,1256677221,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
1,1,481,3.5,1256677456,Kalifornia (1993),Drama|Thriller
2,1,1091,1.5,1256677471,Weekend at Bernie's (1989),Comedy
3,1,1257,4.5,1256677460,Better Off Dead... (1985),Comedy|Romance
4,1,1449,4.5,1256677264,Waiting for Guffman (1996),Comedy


In [29]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [30]:
movie_genres = [change_string(g) for g in df_user.genres.values]

In [31]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(movie_genres)

In [32]:
df1 = pd.DataFrame(X_train_tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
res = pd.concat([df_user, df1], axis=1)

In [33]:
res.head()

,userId,movieId,rating,timestamp,title,genres,action,adventure,comedy,crime,drama,fantasy,horror,romance,scifi,thriller,western
0,1,307,3.5,1256677221,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.00000,0.0,0.000000,0.0
1,1,481,3.5,1256677456,Kalifornia (1993),Drama|Thriller,0.0,0.0,0.000000,0.0,0.732546,0.0,0.0,0.00000,0.0,0.680717,0.0
2,1,1091,1.5,1256677471,Weekend at Bernie's (1989),Comedy,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0
3,1,1257,4.5,1256677460,Better Off Dead... (1985),Comedy|Romance,0.0,0.0,0.568013,0.0,0.000000,0.0,0.0,0.82302,0.0,0.000000,0.0
4,1,1449,4.5,1256677264,Waiting for Guffman (1996),Comedy,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0


In [34]:
y = res['rating']
X = res.drop(['rating', 'title', 'genres', 'userId', 'movieId'], axis=1)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [36]:
X_train.shape

(11, 12)

In [37]:
# Обучаем модель
from sklearn.neighbors import KNeighborsRegressor

In [38]:
model = KNeighborsRegressor(n_neighbors=10)
model.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                    weights='uniform')

In [39]:
y_pred = model.predict(X_test)

In [40]:
from sklearn import metrics
print(f'RMSE: {np.sqrt(metrics.mean_squared_error(y_test, y_pred))}')

RMSE: 1.574801574802362
